In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

folder_path3 = "C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/merged_dataset"


In [3]:
train_test_yearly = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_2/train_test_yearly.csv", index_col=False)
Apports_NPK_yearly = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_2/Apports_NPK_yearly.csv", index_col=False)
irrigation_yearly = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_2/irrigation_yearly.csv", index_col=False)


# Rename the column
irrigation_yearly.rename(columns={'Code Traçabilité': 'CodeTracabilite'}, inplace=True)

# Save the updated DataFrame back to a CSV file
irrigation_yearly.to_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_2/irrigation_yearly.csv", index=False)

In [4]:
train_test_yearly.isnull().sum()  ##none null value

Year                  0
Campagne              0
Region                0
Site                  0
Famille               0
Variete               0
Num Parcelle          0
CodeTracabilite       0
is_train              0
yearly_sum_tonnage    0
dtype: int64

In [5]:
irrigation_yearly.isnull().sum()

Year                       0
Campagne                   0
Region                     0
Site                       0
Famille                    0
Variete                    0
Num Parcelle               0
CodeTracabilite            0
yearly_sum_of_qte_d_eau    0
dtype: int64

In [6]:
train_test_yearly.shape

(2278, 10)

In [7]:
unique_rows = irrigation_yearly.drop_duplicates(subset=['Num Parcelle', 'CodeTracabilite', 'Variete'])
unique_rows

,Year,Campagne,Region,Site,Famille,Variete,Num Parcelle,CodeTracabilite,yearly_sum_of_qte_d_eau
0,2020,20/21,GHARB,Ben Mansour,AF,AF1,11240,TR11240,3341.7000
1,2020,20/21,GHARB,Ben Mansour,CLA,CLA14,11211,TR11211,858.2684
2,2020,20/21,GHARB,Ben Mansour,CLA,CLA16,11160,TR11160,936.0316
3,2020,20/21,GHARB,Ben Mansour,ORA,ORA2,11170,TR11170,2142.0000
4,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA1,13030,TR11130,302.9532
...,...,...,...,...,...,...,...,...,...
3408,2022,22/23,TADLA,Bni Ayat,SPA,SPA1,20108.0,TR20108,314.0113
3828,2023,22/23,ORIENTAL,Cherraa,CLA,CLA14,11055.0,TR11055,2099.2200
3829,2023,22/23,ORIENTAL,Cherraa,CLA,CLA14,11056.0,TR11056,2546.3570
3965,2023,22/23,SOUSS,Taroudant,AF,AF3,18310.0,TR18310,977.8914


In [8]:
unique_rows = Apports_NPK_yearly.drop_duplicates(subset=['Num Parcelle', 'CodeTracabilite', 'Variete'])
unique_rows

,Year,Campagne,Region,Site,Famille,Variete,Num Parcelle,CodeTracabilite,yearly_sum_N,yearly_sum_P,yearly_sum_K
0,2020,20/21,GHARB,Ben Mansour,AF,AF1,11240,TR11240,3.900000,0.000000,13.800000
1,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA1,13030,TR11130,20.347619,4.645285,0.000000
2,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA12,13010,TR11120,29.654580,6.770029,0.000000
3,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA14,12010,TR11020,9.232091,0.000000,0.023421
4,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA14,12010,TR11090,90.554194,30.871385,0.348981
...,...,...,...,...,...,...,...,...,...,...,...
3564,2023,22/23,TADLA,Bni Ayat,MAD,MAD1,60120.0,TR60200,322.617251,40.763284,467.216450
3565,2023,22/23,TADLA,Bni Ayat,MAD,MAD1,60130.0,TR60200,73.466190,9.280081,106.460912
3571,2023,22/23,TADLA,Bni Ayat,ORA,ORA12,10120.0,TR10120,222.678926,53.178376,232.801793
3572,2023,22/23,TADLA,Bni Ayat,ORA,ORA12,10215.0,TR10700,314.972072,72.739201,316.695238


In [9]:
# Merge the datasets
# First, merge 'train_test_yearly' and 'Apports_NPK_yearly'
merged_1 = pd.merge(irrigation_yearly, Apports_NPK_yearly, 
                    how='outer', 
                    left_on=['CodeTracabilite', 'Variete', 'Year', 'Campagne', 'Site', 'Famille', 'Num Parcelle', 'Region'], 
                    right_on=['CodeTracabilite', 'Variete', 'Year', 'Campagne', 'Site', 'Famille', 'Num Parcelle', 'Region'])

# Now, merge the result with 'irrigation_yearly'
test_train_merged = pd.merge(train_test_yearly, merged_1, 
                        how='left', 
                        left_on=['CodeTracabilite', 'Variete', 'Year', 'Campagne', 'Site', 'Famille', 'Num Parcelle', 'Region'], 
                        right_on=['CodeTracabilite', 'Variete', 'Year', 'Campagne', 'Site', 'Famille', 'Num Parcelle', 'Region'])

# Check the first few rows of the final merged dataset
test_train_merged.tail()

,Year,Campagne,Region,Site,Famille,Variete,Num Parcelle,CodeTracabilite,is_train,yearly_sum_tonnage,yearly_sum_of_qte_d_eau,yearly_sum_N,yearly_sum_P,yearly_sum_K
2274,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10403,TR10500,False,0.0,11621.7513,NaN,NaN,NaN
2275,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10406,TR10600,False,0.0,8138.4881,NaN,NaN,NaN
2276,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10407,TR10500,False,0.0,8528.9743,NaN,NaN,NaN
2277,2023,22/23,TADLA,Bni Ayat,ORA,ORA9,20106,TR20400,False,0.0,6196.2969,NaN,NaN,NaN
2278,2023,22/23,TADLA,Bni Ayat,SPA,SPA1,20108,TR20108,False,0.0,NaN,NaN,NaN,NaN
